In [1]:
import purly

In [2]:
from example_utils import localhost
purly.state.Machine().daemon()

# name the layout resource "color-wheel" and connect to the update stream
websocket_url = localhost('ws', 8000) + '/model/color-wheel/stream'

[2018-07-29 14:38:29 -0700] [1476] [INFO] Goin' Fast @ http://127.0.0.1:8000
[2018-07-29 14:38:29 -0700] [1476] [INFO] Starting worker [1476]
[2018-07-29 14:38:30 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/color-wheel/assets/index.html  200 573
[2018-07-29 14:38:30 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/color-wheel/assets/static/css/main.785d0ef8.css  200 118
[2018-07-29 14:38:30 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/color-wheel/assets/static/js/main.6da5cf05.js  200 192350
[2018-07-29 14:38:30 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/color-wheel/assets/static/css/main.785d0ef8.css.map  200 299
[2018-07-29 14:38:30 -0700] - (sanic.access)[INFO][1:2]: GET http://127.0.0.1:8000/model/color-wheel/assets/static/js/main.6da5cf05.js.map  200 1731473
[2018-07-29 14:38:35 -0700] [1476] [INFO] KeepAlive Timeout. Closing connection.
[2018-07-29 14:38:35 -0700] [1476] [INFO] KeepAlive Ti

Error in data transfer
Traceback (most recent call last):
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/websockets/protocol.py", line 669, in write_frame
    yield from self.writer.drain()
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/asyncio/streams.py", line 339, in drain
    yield from self._protocol._drain_helper()
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/asyncio/streams.py", line 210, in _drain_helper
    raise ConnectionResetError('Connection lost')
ConnectionResetError: Connection lost

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/websockets/protocol.py", line 496, in transfer_data
    msg = yield from self.read_message()
  File "/home/ryan/conda/envs/Py3.6/lib/python3.6/site-packages/websockets/protocol.py", line 526, in read_message
    frame = yield from self.read_data_frame(max_size=self.max_size)
  File "/home/ryan/conda/e

In [3]:
websocket_url

'ws://127.0.0.1:8000/model/color-wheel/stream'

In [4]:
layout = purly.Layout(websocket_url)

In [5]:
def dot(radius, color, margin):
    return layout.html('div', style=dict(
        height="%spx" % (radius * 2),
        width="%spx" % (radius * 2),
        backgroundColor=color,
        borderRadius="50%",
        margin=str(margin) + 'px',
        display='inline-block'
    ))


def grid(x, y, size):
    container = layout.html('div')
    for i in range(y):
        row = layout.html('div')
        for j in range(x):
            d = dot(size, 'blue', size / 2)
            row.children.append(d)
        container.children.append(row)
    return container

In [6]:
circle_size = 7

g = grid(10, 5, circle_size)

In [7]:
layout.children.clear()
layout.children.append(g)
layout.sync()
layout

In [ ]:
drag = {}
selection = set()


def select(drag):
    y_start, y_end = sorted([
        drag['start']['y'] // (circle_size * 3),
        drag['end']['y'] // (circle_size * 3),
    ])
    x_start, x_end = sorted([
        drag['start']['x'] // (circle_size * 3),
        drag['end']['x'] // (circle_size * 3),
    ])
    to_select = set()
    for i in range(y_start, y_end):
        try:
            row = g.children[i]
        except IndexError:
            break
        for j in range(x_start, x_end):
            try:
                new = row.children[j]
            except IndexError:
                break
            else:
                to_select.add(new)
    for element in selection.difference(to_select):
        element.style['backgroundColor'] = 'blue'
        if element in selection:
            selection.remove(element)
    for element in to_select.difference(selection):
        element.style['backgroundColor'] = 'red'
        selection.add(element)


@g.on("DragStart")
def drag_start(clientX, clientY):    
    drag['start'] = {'x': clientX, 'y': clientY}


@g.on("Drag")
def dragging(clientX, clientY):
    drag['end'] = {'x': clientX, 'y': clientY}
    select(drag.copy())

@g.on("DragEnd")
def drag_end(clientX, clientY):
    drag.clear()



layout.serve()